## Detector de SPAM

O intuito deste projeto é desenvolver um modelo de machine learning que possua a capacidade de identificar se uma determinada

mensagem é um SPAM.

O termo Spam pode ser um acrónimo derivado da expressão em inglês "Sending and Posting Advertisement in Mass", traduzido em português 

"Enviar e Postar Publicidade em Massa", Normalmente são mensagens eletrônicas que recebemos por e-mail, SMS, Redes sociais sem o nosso 

consentimento.

OBS: Este projeto poderia ser desenvolvido somente com pyhton, scikit-learn e NLTK, porém desta vez prefiri fazer uso do apache SPARK.

DATASET retirado do site Kaggle, endereço:

https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [1]:
# Imports utilizados
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IDF, HashingTF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.pipeline import Pipeline

In [2]:
# verificando a versão do spark
sc.version

'2.4.2'

In [3]:
# Criando uma sessão spark session para usarmos sparkSQL
spSession = SparkSession.builder.master("Local").appName("SpamDetect").getOrCreate()

In [4]:
# Carregando os registros de SMS e mostrando as primeiras linhas 
spamdata = sc.textFile("spam.csv")
spamdata.take(5)

['v1,v2,,,',
 'ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,',
 'ham,Ok lar... Joking wif u oni...,,,',
 "spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,",
 'ham,U dun say so early hor... U c already then say...,,,']

In [5]:
# Quantidade de registros
spamdata.count()

5575

In [6]:
# Removendo o cabeçalho
header = spamdata.first()
spamdataRDD = spamdata.filter(lambda line : line != header)
spamdataRDD.take(2)

['ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,',
 'ham,Ok lar... Joking wif u oni...,,,']

In [7]:
spamdataRDD.count()

5574

### Limpeza e pré-processamento dos dados

In [8]:
# Transformando o atributo que classifica se o SMS é um spam para formato numérico
def transformWords(rdd):
    attList = rdd.split(',')
    smsMessage = 1.0 if attList[0] == 'spam' else 0.0
    return [smsMessage, attList[1]]

In [9]:
# Aplicando a função ao RDD
# Lembrando que uma RDD é imutável

spamdataRDD2 = spamdataRDD.map(transformWords)
spamdataRDD2.take(10)

[[0.0, '"Go until jurong point'],
 [0.0, 'Ok lar... Joking wif u oni...'],
 [1.0,
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
 [0.0, 'U dun say so early hor... U c already then say...'],
 [0.0, '"Nah I don\'t think he goes to usf'],
 [1.0,
  '"FreeMsg Hey there darling it\'s been 3 week\'s now and no word back! I\'d like some fun you up for it still? Tb ok! XxX std chgs to send'],
 [0.0,
  'Even my brother is not like to speak with me. They treat me like aids patent.'],
 [0.0,
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune"],
 [1.0,
  'WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'],
 [1.0,
  'Had your mobile 11 months or more? U R entitled to U

In [10]:
# Criando um dataframe com o SparkSQL
# Lembrando que estou usando o JDK8, pois o 11 ainda não possui suporte para trabalharmos com o modelo ANSI (Instruções SQL originais), no qual eu 
# particularmente prefiro

spamDF = spSession.createDataFrame(spamdataRDD2, ['label', 'message'])
spamDF.createOrReplaceTempView("tableSpam")
spSession.sql("Select label, message from tableSpam").show()

+-----+--------------------+
|label|             message|
+-----+--------------------+
|  0.0|"Go until jurong ...|
|  0.0|Ok lar... Joking ...|
|  1.0|Free entry in 2 a...|
|  0.0|U dun say so earl...|
|  0.0|"Nah I don't thin...|
|  1.0|"FreeMsg Hey ther...|
|  0.0|Even my brother i...|
|  0.0|As per your reque...|
|  1.0|WINNER!! As a val...|
|  1.0|Had your mobile 1...|
|  0.0|"I'm gonna be hom...|
|  1.0|"SIX chances to w...|
|  1.0|"URGENT! You have...|
|  0.0|I've been searchi...|
|  0.0|I HAVE A DATE ON ...|
|  1.0|"XXXMobileMovieCl...|
|  0.0|Oh k...i'm watchi...|
|  0.0|Eh u remember how...|
|  0.0|Fine if that��s t...|
|  1.0|"England v Macedo...|
+-----+--------------------+
only showing top 20 rows



### Processamento de Linguagem natural

<p>Como estamos lidando com linguagem verbal escrita, precisamos aplicar alguns tratamento para melhorar
a qualidade de aprendizado do algoritmo de machine learning. Logo iremos aplicar alguns métodos de processamento de linguagem natural, como 
tokenização, remoção das stopwords e stemming.</p>

<p>Vamos a algumas explicações antes, tokenização é o nome dado ao processo que se refere a separar textos em "pedaços", tokens, por exemplo, caso 
tenhamos um parágrafo posso tokenizar em sentenças e a partir das sentenças tokenizar em palavras.</p>

<p>StopWords são aquelas palavras de um respectivo idioma no qual não agrega muito significado na sentença, pelo menos em relação a uma análise e para
otimizar a performance do modelo normalmente as removemos. Exemplos de stopwords em português são as palavras, de, da, um, umas.</p>

<p>Stemmings são os sufixos e prefixos de palavras que também removemos para otimização do modelo, o Google costuma utilizar esta técnica no momento de 
realizar as suas buscas.</p>

In [11]:
# Removendo os stemmings das palavras

spamDF2 = spamDF.select("label", (regexp_replace('message', 'ing|er|ier|est', " ")))
spamDF2 = spamDF2.toPandas()
spamDF3 = spSession.createDataFrame(spamDF2, ['label', 'message'])
spamDF3.select("*").show()

+-----+--------------------+
|label|             message|
+-----+--------------------+
|  0.0|"Go until jurong ...|
|  0.0|Ok lar... Jok  wi...|
|  1.0|Free entry in 2 a...|
|  0.0|U dun say so earl...|
|  0.0|"Nah I don't thin...|
|  1.0|"FreeMsg Hey th e...|
|  0.0|Even my broth  is...|
|  0.0|As p  your requ  ...|
|  1.0|WINNER!! As a val...|
|  1.0|Had your mobile 1...|
|  0.0|"I'm gonna be hom...|
|  1.0|"SIX chances to w...|
|  1.0|"URGENT! You have...|
|  0.0|I've been search ...|
|  0.0|I HAVE A DATE ON ...|
|  1.0|"XXXMobileMovieCl...|
|  0.0|Oh k...i'm watch ...|
|  0.0|Eh u rememb  how ...|
|  0.0|Fine if that��s t...|
|  1.0|"England v Macedo...|
+-----+--------------------+
only showing top 20 rows



In [12]:
# HashingTF cria uma tabela com a frequencia de cada termo remanescente e o IDF cálcula estatisticamente a importância
# de cada palavra com base na frequência criada pelo hashingTF.
tokenizer = Tokenizer(inputCol='message', outputCol='tokenMessage')
stopWords = StopWordsRemover(inputCol= tokenizer.getOutputCol(), outputCol= 'outStopWords')
hashingTF = HashingTF(inputCol= stopWords.getOutputCol(), outputCol= "hashingWords")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol= "features")

In [13]:
# String Indexer é utilizado para converter variáveis categóricas, contudo, é também um requerimento para podermos
# testar algoritmos de ML baseado em árvores (trees).
stringindexer = StringIndexer(inputCol= 'label', outputCol='indexed')
si_model = stringindexer.fit(spamDF3)
spamDF4 = si_model.transform(spamDF3)

spamDF4.select("label", 'message', 'indexed').show()

+-----+--------------------+-------+
|label|             message|indexed|
+-----+--------------------+-------+
|  0.0|"Go until jurong ...|    0.0|
|  0.0|Ok lar... Jok  wi...|    0.0|
|  1.0|Free entry in 2 a...|    1.0|
|  0.0|U dun say so earl...|    0.0|
|  0.0|"Nah I don't thin...|    0.0|
|  1.0|"FreeMsg Hey th e...|    1.0|
|  0.0|Even my broth  is...|    0.0|
|  0.0|As p  your requ  ...|    0.0|
|  1.0|WINNER!! As a val...|    1.0|
|  1.0|Had your mobile 1...|    1.0|
|  0.0|"I'm gonna be hom...|    0.0|
|  1.0|"SIX chances to w...|    1.0|
|  1.0|"URGENT! You have...|    1.0|
|  0.0|I've been search ...|    0.0|
|  0.0|I HAVE A DATE ON ...|    0.0|
|  1.0|"XXXMobileMovieCl...|    1.0|
|  0.0|Oh k...i'm watch ...|    0.0|
|  0.0|Eh u rememb  how ...|    0.0|
|  0.0|Fine if that��s t...|    0.0|
|  1.0|"England v Macedo...|    1.0|
+-----+--------------------+-------+
only showing top 20 rows



### Aplicando Machine Learning

In [14]:
# Dividindo o DataFrame em conjuntos de dados treino e teste
dados_treino, dados_teste = spamDF4.randomSplit([0.8,0.2], seed= 7)

In [15]:
dados_treino.count()

4454

In [16]:
dados_teste.count()

1120

### Criação dos PipeLines

<p>Normalmente para verificação de Spams o algoritmo mais recomendado é o Naive Bayes, pois ele é rápido, simples e 
retorna a probabilidade do resultado. </p>

<p>Contudo também irei testar com os modelos de árvores, como decision tree e random forest, ambos somente aceitam váriaveis do tipo numérica, como neste caso são mensagens eu usei o atributo gerado a partir do modulo IDF explicado anteriormente</p>

In [17]:
naiveBayes = NaiveBayes()
dcTree = DecisionTreeClassifier(maxDepth= 10, labelCol='indexed', featuresCol= 'features')
rdforest = RandomForestClassifier(labelCol="indexed", featuresCol='features', numTrees= 10)

# Pipelines
pipeDcTree = Pipeline(stages=[tokenizer, stopWords, hashingTF, idf, dcTree])
pipeNB = Pipeline(stages=[tokenizer, stopWords, hashingTF, idf, naiveBayes])
pipeRForest = Pipeline(stages=[tokenizer, stopWords, hashingTF, idf, rdforest])

In [18]:
# Treinando o modelo Naive Bayes
model = pipeNB.fit(dados_treino)
previsoes = model.transform(dados_teste)

In [19]:
# Treinando o modelo de árvore de decisão
model1 = pipeDcTree.fit(dados_treino)
previsoes1 = model1.transform(dados_teste)

In [20]:
# Treinando o modelo Random Forest
model2 = pipeRForest.fit(dados_treino)
previsoes2 = model2.transform(dados_teste)

<p>Imprimindo os dez primeiros Registros com cada previsão realizada</p>

In [21]:
# Dez primeiras previsões do modelo Random Forest
previsoes2.select("label", "prediction", "message").show(10)

+-----+----------+--------------------+
|label|prediction|             message|
+-----+----------+--------------------+
|  0.0|       0.0|                    |
|  0.0|       0.0|      "Aight will do|
|  0.0|       0.0|        "Alright omw|
|  0.0|       0.0|"Although i told ...|
|  0.0|       0.0|            "Awesome|
|  0.0|       0.0|"Beautiful Truth ...|
|  0.0|       0.0|         "Come to mu|
|  0.0|       0.0|               "Cool|
|  0.0|       0.0|               "Dear|
|  0.0|       0.0|               "Dear|
+-----+----------+--------------------+
only showing top 10 rows



In [22]:
# Dez primeiras previsões do modelo Decision Tree
previsoes1.select("label", "prediction", "message").show(10)

+-----+----------+--------------------+
|label|prediction|             message|
+-----+----------+--------------------+
|  0.0|       0.0|                    |
|  0.0|       0.0|      "Aight will do|
|  0.0|       0.0|        "Alright omw|
|  0.0|       0.0|"Although i told ...|
|  0.0|       0.0|            "Awesome|
|  0.0|       0.0|"Beautiful Truth ...|
|  0.0|       0.0|         "Come to mu|
|  0.0|       0.0|               "Cool|
|  0.0|       0.0|               "Dear|
|  0.0|       0.0|               "Dear|
+-----+----------+--------------------+
only showing top 10 rows



In [23]:
# Dez primeiras previsões do modelo Naive Bayes
previsoes.select("label", "prediction", "message").show(10)

+-----+----------+--------------------+
|label|prediction|             message|
+-----+----------+--------------------+
|  0.0|       0.0|                    |
|  0.0|       0.0|      "Aight will do|
|  0.0|       0.0|        "Alright omw|
|  0.0|       0.0|"Although i told ...|
|  0.0|       0.0|            "Awesome|
|  0.0|       0.0|"Beautiful Truth ...|
|  0.0|       0.0|         "Come to mu|
|  0.0|       0.0|               "Cool|
|  0.0|       0.0|               "Dear|
|  0.0|       0.0|               "Dear|
+-----+----------+--------------------+
only showing top 10 rows



### Verificando a acurácia dos algoritmos

In [24]:
# Naive Bayes
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName= 'accuracy')
print(evaluator.evaluate(previsoes))

# Decision Tree
evaluator1 = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName= 'accuracy')
print(evaluator1.evaluate(previsoes1))

# Random Forest
evaluator2 = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName= 'accuracy')
print(evaluator1.evaluate(previsoes2))

0.9598214285714286
0.9267857142857143
0.8580357142857142


<p>Podemos verificar que realmente o algoritmo Naive Bayes se sobressaiu em relação aos demais com uma taxa de acerto
equivalente a 96% , ou seja, a cada cem mensagens ele acerta 96, é uma ótima acurácia.</p>

### Confusion Matrix

In [25]:
# Naive Bayes
previsoes.groupby("label", 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  136|
|  0.0|       1.0|   22|
|  1.0|       0.0|   23|
|  0.0|       0.0|  939|
+-----+----------+-----+



In [26]:
# Decision Tree
previsoes1.groupby("label", 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   85|
|  0.0|       1.0|    8|
|  1.0|       0.0|   74|
|  0.0|       0.0|  953|
+-----+----------+-----+



In [27]:
# Random Forest
previsoes2.groupby("label", 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       0.0|  159|
|  0.0|       0.0|  961|
+-----+----------+-----+



Com isso concluimos nosso projeto para detecção de spams com êxito.